<a href="https://colab.research.google.com/github/shashithenuwara/IRWA_Project/blob/frontend/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2 pdfplumber spacy nltk whoosh transformers torch beautifulsoup4
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from PyPDF2 import PdfReader

# Path to the dataset
pdf_directory = '/content/drive/My Drive/Colab Notebooks/MIniDataSet'

# Iterate through all PDFs in the directory and extract text
text_data = ""
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, filename)
        with open(pdf_path, 'rb') as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text_data += page.extract_text() + "\n"

# Now 'text_data' contains the combined text of all the PDFs


In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')

# Tokenization
tokens = word_tokenize(text_data)

# Normalization (Lowercasing, Removing punctuation, Stopwords)
tokens = [word.lower() for word in tokens if word.isalpha()]
tokens = [word for word in tokens if word not in stopwords.words('english')]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import spacy

nlp = spacy.load('en_core_web_sm')

# Increase the max_length limit
nlp.max_length = len(text_data)

doc = nlp(text_data)

for ent in doc.ents:
    print(ent.text, ent.label_)

Streaming output truncated to the last 5000 lines.
479–484 CARDINAL
Lu PERSON
Weaver GPE
V.M. ORG
Werb, Z. ( ORG
2012 DATE
J. Cell Biol PERSON
196 CARDINAL
395–406 CARDINAL
Luria PERSON
S.E. PERSON
Delbru PERSON
1943 DATE
Virus FAC
Virus Resistance ORG
28 CARDINAL
491–511 CARDINAL
Malikic GPE
McPherson ORG
A.W. PERSON
Donmez PERSON
Sahinalp GPE
C.S. GPE
2015 DATE
1349–1356 CARDINAL
Marcucci GPE
F. PERSON
Stassi GPE
G. PERSON
De Maria PERSON
R. NORP
2016 DATE
Drug Discov PERSON
15 CARDINAL
311–325 CARDINAL
Martelotto GPE
L.G. GPE
Baslan GPE
T. GPE
Kendall GPE
J., Geyer ORG
F.C. GPE
Burke GPE
K.A. GPE
Spraggon GPE
L. ORG
Piscuoglio PRODUCT
Chadalavada GPE
K. NORP
Nanjangud PERSON
Ng GPE
C.K. GPE
23 CARDINAL
376–385 CARDINAL
McKenna PERSON
A. PERSON
Hanna GPE
M., Banks, E., ORG
Sivachenko PERSON
A. PERSON
Cibulskis PERSON
K. NORP
Kernytsky GPE
Garimella ORG
K. NORP
Altshuler ORG
D. NORP
Gabriel PERSON
DePristo ORG
M.A. GPE
2010).The Genome Analysis Toolkit ORG
MapReduce ORG
Genome Res ORG

In [6]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Stemming
stemmer = PorterStemmer()
stems = [stemmer.stem(word) for word in tokens]

# Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(word) for word in tokens]


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT
import os

# Define schema
schema = Schema(title=TEXT(stored=True), content=TEXT)

# Create index directory
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

# Create an index
ix = create_in("indexdir", schema)

# Indexing documents (Assuming titles are available for each document)
writer = ix.writer()

# Add documents to index
writer.add_document(title="Document 1", content="This is the first document.")
writer.add_document(title="Document 2", content="This is the second document.")
writer.commit()


In [19]:
# Indexing documents
writer = ix.writer()
# Add documents to index (replace with your actual PDF titles and content)
for i, filename in enumerate(os.listdir(pdf_directory)):
    if filename.endswith(".pdf"):
        with open(os.path.join(pdf_directory, filename), 'rb') as file:
            reader = PdfReader(file)
            content = ""
            for page in reader.pages:
                content += page.extract_text() + "\n"
            writer.add_document(title=filename, content=content)
writer.commit()

In [20]:
from whoosh.qparser import QueryParser
from whoosh.index import open_dir # import the open_dir function

ix = open_dir("indexdir") # Call the correct function
with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse("document")
    results = searcher.search(query)
    for result in results:
        print(result['title'])

Document 1
Document 2


In [21]:
from nltk.corpus import wordnet

# Get synonyms for query expansion
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return set(synonyms)

query = "happy"
expanded_query = query + " " + " ".join(get_synonyms(query))
print(expanded_query)


happy felicitous happy well-chosen glad


In [22]:
!pip install sentence_transformers

In [23]:
from sentence_transformers import SentenceTransformer, util

# Load BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Semantic search
query = "What is AI?"
query_embedding = model.encode(query)

documents = ["Document about artificial intelligence.", "Text related to machine learning."]
doc_embeddings = model.encode(documents)

# Find best match
hits = util.semantic_search(query_embedding, doc_embeddings)
print(hits)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[[{'corpus_id': 0, 'score': 0.5344616174697876}, {'corpus_id': 1, 'score': 0.22655534744262695}]]


In [24]:
from whoosh import query

ix = open_dir("indexdir")
with ix.searcher() as searcher:
    fuzzy_query = query.FuzzyTerm("content", "document", maxdist=1)
    results = searcher.search(fuzzy_query)
    for result in results:
        print(result['title'])


Document 1
Document 2


In [25]:
!pip install streamlit
import streamlit as st
from whoosh.qparser import QueryParser
from whoosh.index import open_dir

# Streamlit app for search engine
st.title("Search Engine")

query = st.text_input("Enter search query:")

if query:
    ix = open_dir("indexdir")
    with ix.searcher() as searcher:
        parser = QueryParser("content", ix.schema)
        parsed_query = parser.parse(query)
        results = searcher.search(parsed_query)

        for result in results:
            st.write(result['title'])

2024-09-29 09:05:05.006 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 09:05:05.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 09:05:05.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 09:05:05.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 09:05:05.026 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 09:05:05.030 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-29 09:05:05.034 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have predicted and actual results
y_true = [1, 0, 1, 1, 0]
y_pred = [1, 0, 1, 0, 0]

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f'Precision: {precision}, Recall: {recall}, F1-Score: {f1}')


Precision: 1.0, Recall: 0.6666666666666666, F1-Score: 0.8


In [27]:
!pip install streamlit pyngrok joblib


In [28]:
%%writefile app.py

import streamlit as st
from whoosh.index import open_dir
from whoosh.qparser import QueryParser

# Define the main function for the Streamlit app
def main():
    st.title("Search Engine")

    query = st.text_input("Enter your search query:")

    if st.button("Search"):
        if query:
            # Perform search
            ix = open_dir("indexdir")
            with ix.searcher() as searcher:
                parser = QueryParser("content", ix.schema)
                parsed_query = parser.parse(query)
                results = searcher.search(parsed_query)

                st.write("Search Results:")
                for result in results:
                    st.write(result['title'])
        else:
            st.write("Please enter a query to search.")

if __name__ == '__main__':
    main()

Overwriting app.py


In [29]:
from pyngrok import ngrok
import os

# Kill any existing Streamlit processes
os.system('pkill streamlit')

# Run the Streamlit app
get_ipython().system_raw('streamlit run app.py &')

# Create a public URL using ngrok
!ngrok authtoken 2mbYowCbzDDA47omhyfn7mfDSnU_7RKJtcLWCysu7WCAV5Whj
url = ngrok.connect(addr='http://localhost:8501')
print(f"Streamlit app is live at: {url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app is live at: NgrokTunnel: "https://0749-34-136-108-55.ngrok-free.app" -> "http://localhost:8501"
